In [1]:
import pandas as pd
import optuna
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import (
    roc_curve, roc_auc_score, accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, precision_recall_curve, auc
)
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import random

warnings.filterwarnings('ignore')

In [29]:
# 데이터 불러오기

per = 100

train_data = pd.read_csv("/home/watercar99/project/MIMIC-III/Data/FINAL/FINAL_Split/Final_train_data("+str(per)+"%)_down.csv")
test_data = pd.read_csv("/home/watercar99/project/MIMIC-III/Data/FINAL/FINAL_Split/Final_test_data("+str(per)+"%)_down.csv")

train_data.drop(["SUBJECT_ID","HADM_ID", "DOA", "ETHNICITY"], axis=1, inplace=True)
test_data.drop(["SUBJECT_ID","HADM_ID", "DOA",  "ETHNICITY"], axis=1, inplace=True)
                
train_data = pd.get_dummies(train_data, columns=['GENDER', 'Ventilator', 'Anisocytosis', 'Macrocytes', 'Poikilocytosis', 'Bacteria', 'Bilirubin', 'Urine Appearance', 'Urine Color'])
test_data = pd.get_dummies(test_data, columns=['GENDER', 'Ventilator', 'Anisocytosis', 'Macrocytes', 'Poikilocytosis', 'Bacteria', 'Bilirubin', 'Urine Appearance', 'Urine Color'])

In [30]:
# train_data와 test_data의 칼럼을 맞추기 위해 차집합을 계산
missing_columns_in_test = set(train_data.columns) - set(test_data.columns)
missing_columns_in_train = set(test_data.columns) - set(train_data.columns)

# test_data에 train_data의 더미 변수를 추가하고 0으로 채워주기
for col in missing_columns_in_test:
    test_data[col] = 0

# train_data에 test_data의 더미 변수를 추가하고 0으로 채워주기
for col in missing_columns_in_train:
    train_data[col] = 0

# Features와 Target 설정
X_train = train_data.drop('y', axis=1)
y_train = train_data['y']
X_test = test_data.drop('y', axis=1)

print("train shape\n",X_train.shape,"\n")
print("train shape\n",X_test.shape,"\n")

target = "y"
features = [f for f in train_data.columns if f not in [target]]

print("train value\n",train_data['y'].value_counts())
print("test value\n",test_data['y'].value_counts())

train shape
 (978, 952) 

train shape
 (457, 952) 

train value
 1    489
0    489
Name: y, dtype: int64
test value
 0    341
1    116
Name: y, dtype: int64


In [31]:
def objective(trial):
    c = trial.suggest_float("c", 1e-8, 10.0, log=True)
    kernel = trial.suggest_categorical("kernel", ["poly"])
    
    model = SVC(C=c, kernel=kernel, probability=True, decision_function_shape="ovo", random_state=42)
    
    # Cross Validation을 통한 평가 지표 계산
    scores = cross_val_score(model, X_train, y_train, cv=3, scoring='f1')
    return scores.mean()

In [32]:
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []
auroc_list = []
y_pred_list = []

for _ in range(10):
    # Optuna 스터디 설정
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=30, gc_after_trial=True, n_jobs=-1)

    # 최적 하이퍼파라미터 확인
    best_trial = study.best_trial
    best_c = best_trial.params["c"]
    best_kernel = best_trial.params["kernel"]

    # 최적 모델 학습 및 평가
    best_model = SVC(C=best_c, kernel=best_kernel, random_state=42 , probability=True)
    best_model.fit(X_train, y_train)
    
    # Test 데이터로 모델 평가 및 예측
    y_pred = best_model.predict(X_test)
    y_score = best_model.predict_proba(X_test)
    accuracy = accuracy_score(test_data['y'], y_pred)
    precision = precision_score(test_data['y'], y_pred)
    recall = recall_score(test_data['y'], y_pred)
    f1 = f1_score(test_data['y'], y_pred)
    auroc = roc_auc_score(test_data['y'], y_score[:, 1])

    rounded_accuracy = round(accuracy * 100, 2)
    rounded_precision = round(precision * 100, 2)
    rounded_recall = round(recall * 100, 2)
    rounded_f1 = round(f1 * 100, 2)
    rounded_auroc = round(auroc * 100, 2)

    # 결과 및 예측값을 리스트에 추가
    accuracy_list.append(rounded_accuracy)
    precision_list.append(rounded_precision)
    recall_list.append(rounded_recall)
    f1_list.append(rounded_f1)
    auroc_list.append(rounded_auroc)
    y_pred_list.append(y_pred)

    print(f"Iteration {_ + 1} Results:")
    print("Best Trial Parameters:")
    print("c:", best_c)
    print("kernel:", best_kernel)
    print("Test Accuracy:", rounded_accuracy)
    print("Precision:", rounded_precision)
    print("Recall:", rounded_recall)
    print("F1-score:", rounded_f1)
    print("AUROC:", rounded_auroc)
    print("")

# 결과 출력
print("Mean Accuracy:", sum(accuracy_list) / len(accuracy_list))
print("Mean Precision:", sum(precision_list) / len(precision_list))
print("Mean Recall:", sum(recall_list) / len(recall_list))
print("Mean F1-score:", sum(f1_list) / len(f1_list))
print("Mean AUROC:", sum(auroc_list) / len(auroc_list))

# 예측값 리스트 출력
for i, y_pred in enumerate(y_pred_list):
    print(f"Iteration {i + 1} Predictions:")
    print(y_pred)


[I 2023-10-05 22:16:10,496] A new study created in memory with name: no-name-3f3fa31d-af32-4364-bf01-c2cdb6e2d49d
[I 2023-10-05 22:16:28,059] Trial 27 finished with value: 0.22131147540983606 and parameters: {'c': 2.126458458955874e-07, 'kernel': 'poly'}. Best is trial 27 with value: 0.22131147540983606.
[I 2023-10-05 22:16:29,549] Trial 6 finished with value: 0.22131147540983606 and parameters: {'c': 7.78766276496046e-08, 'kernel': 'poly'}. Best is trial 27 with value: 0.22131147540983606.
[I 2023-10-05 22:16:30,523] Trial 11 finished with value: 0.22131147540983606 and parameters: {'c': 3.0222113943388333e-07, 'kernel': 'poly'}. Best is trial 27 with value: 0.22131147540983606.
[I 2023-10-05 22:16:30,945] Trial 4 finished with value: 0.22131147540983606 and parameters: {'c': 2.2409253699526224e-06, 'kernel': 'poly'}. Best is trial 27 with value: 0.22131147540983606.
[I 2023-10-05 22:16:31,241] Trial 12 finished with value: 0.5095266032491025 and parameters: {'c': 0.2415730561045929, 

Iteration 1 Results:
Best Trial Parameters:
c: 0.008617938134862375
kernel: poly
Test Accuracy: 25.6
Precision: 25.44
Recall: 100.0
F1-score: 40.56
AUROC: 45.53



[I 2023-10-05 22:16:53,614] Trial 8 finished with value: 0.2677075342568777 and parameters: {'c': 3.7242214715704196, 'kernel': 'poly'}. Best is trial 8 with value: 0.2677075342568777.
[I 2023-10-05 22:16:55,538] Trial 10 finished with value: 0.22130584192439862 and parameters: {'c': 0.0010611131908560135, 'kernel': 'poly'}. Best is trial 8 with value: 0.2677075342568777.
[I 2023-10-05 22:16:55,946] Trial 22 finished with value: 0.24949751900006278 and parameters: {'c': 0.49227614429440664, 'kernel': 'poly'}. Best is trial 8 with value: 0.2677075342568777.
[I 2023-10-05 22:16:56,326] Trial 26 finished with value: 0.22131147540983606 and parameters: {'c': 4.09601859508892e-08, 'kernel': 'poly'}. Best is trial 8 with value: 0.2677075342568777.
[I 2023-10-05 22:16:56,613] Trial 7 finished with value: 0.22131147540983606 and parameters: {'c': 1.1312495811740457e-07, 'kernel': 'poly'}. Best is trial 8 with value: 0.2677075342568777.
[I 2023-10-05 22:16:56,682] Trial 4 finished with value: 0

Iteration 2 Results:
Best Trial Parameters:
c: 0.007463574608650961
kernel: poly
Test Accuracy: 25.6
Precision: 25.44
Recall: 100.0
F1-score: 40.56
AUROC: 45.52



[I 2023-10-05 22:17:19,629] Trial 2 finished with value: 0.2217659137577002 and parameters: {'c': 6.183035602510002e-05, 'kernel': 'poly'}. Best is trial 2 with value: 0.2217659137577002.
[I 2023-10-05 22:17:20,096] Trial 1 finished with value: 0.22131147540983606 and parameters: {'c': 2.5243290064855295e-05, 'kernel': 'poly'}. Best is trial 2 with value: 0.2217659137577002.
[I 2023-10-05 22:17:20,742] Trial 6 finished with value: 0.22131147540983606 and parameters: {'c': 3.659475039865214e-05, 'kernel': 'poly'}. Best is trial 2 with value: 0.2217659137577002.
[I 2023-10-05 22:17:21,176] Trial 16 finished with value: 0.5125826480020291 and parameters: {'c': 0.23993879172073013, 'kernel': 'poly'}. Best is trial 16 with value: 0.5125826480020291.
[I 2023-10-05 22:17:21,875] Trial 4 finished with value: 0.6648168302987703 and parameters: {'c': 0.1824128636872788, 'kernel': 'poly'}. Best is trial 4 with value: 0.6648168302987703.
[I 2023-10-05 22:17:22,364] Trial 23 finished with value: 0.

...

Iteration 8 Results:
Best Trial Parameters:
c: 0.010195584717256602
kernel: poly
Test Accuracy: 25.82
Precision: 25.49
Recall: 100.0
F1-score: 40.63
AUROC: 45.45



[I 2023-10-05 22:19:55,331] Trial 15 finished with value: 0.22131147540983606 and parameters: {'c': 6.65461424156059e-06, 'kernel': 'poly'}. Best is trial 15 with value: 0.22131147540983606.
[I 2023-10-05 22:19:55,811] Trial 10 finished with value: 0.24949751900006278 and parameters: {'c': 0.5583100193865002, 'kernel': 'poly'}. Best is trial 10 with value: 0.24949751900006278.
[I 2023-10-05 22:19:55,821] Trial 0 finished with value: 0.256575900713161 and parameters: {'c': 1.4305226318370712, 'kernel': 'poly'}. Best is trial 0 with value: 0.256575900713161.
[I 2023-10-05 22:19:57,875] Trial 3 finished with value: 0.26725122579235566 and parameters: {'c': 2.708732981317687, 'kernel': 'poly'}. Best is trial 3 with value: 0.26725122579235566.
[I 2023-10-05 22:19:58,302] Trial 18 finished with value: 0.22131147540983606 and parameters: {'c': 2.582255275913055e-08, 'kernel': 'poly'}. Best is trial 3 with value: 0.26725122579235566.
[I 2023-10-05 22:19:58,643] Trial 7 finished with value: 0.2

Iteration 9 Results:
Best Trial Parameters:
c: 0.021437127965354873
kernel: poly
Test Accuracy: 25.6
Precision: 25.33
Recall: 99.14
F1-score: 40.35
AUROC: 43.91



[I 2023-10-05 22:20:20,972] Trial 0 finished with value: 0.2681657244057895 and parameters: {'c': 3.9846756604684965, 'kernel': 'poly'}. Best is trial 0 with value: 0.2681657244057895.
[I 2023-10-05 22:20:22,069] Trial 10 finished with value: 0.6657512213793714 and parameters: {'c': 0.053857903660420745, 'kernel': 'poly'}. Best is trial 10 with value: 0.6657512213793714.
[I 2023-10-05 22:20:22,457] Trial 25 finished with value: 0.665750286368843 and parameters: {'c': 0.04014489477845636, 'kernel': 'poly'}. Best is trial 10 with value: 0.6657512213793714.
[I 2023-10-05 22:20:22,931] Trial 1 finished with value: 0.256575900713161 and parameters: {'c': 1.6617968883816165, 'kernel': 'poly'}. Best is trial 10 with value: 0.6657512213793714.
[I 2023-10-05 22:20:23,808] Trial 2 finished with value: 0.22131147540983606 and parameters: {'c': 1.0258272303265104e-08, 'kernel': 'poly'}. Best is trial 10 with value: 0.6657512213793714.
[I 2023-10-05 22:20:24,325] Trial 5 finished with value: 0.4439

Iteration 10 Results:
Best Trial Parameters:
c: 0.012963331819260107
kernel: poly
Test Accuracy: 25.82
Precision: 25.49
Recall: 100.0
F1-score: 40.63
AUROC: 44.35

Mean Accuracy: 25.688
Mean Precision: 25.427
Mean Recall: 99.74199999999999
Mean F1-score: 40.525
Mean AUROC: 45.20200000000001
Iteration 1 Predictions:
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1

In [33]:
data = {
    'Accuracy': accuracy_list,
    'Precision': precision_list,
    'Recall': recall_list,
    'F1 Score': f1_list,
    'AUROC':auroc_list,
    'Predictions': y_pred_list
    
}

# 딕셔너리를 DataFrame으로 변환
df = pd.DataFrame(data)
df

,Accuracy,Precision,Recall,F1 Score,AUROC,Predictions
0,25.60,25.44,100.00,40.56,45.53,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,25.60,25.44,100.00,40.56,45.52,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,25.82,25.49,100.00,40.63,45.02,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,25.82,25.49,100.00,40.63,45.45,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,25.60,25.44,100.00,40.56,45.55,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
5,25.60,25.33,99.14,40.35,45.39,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
6,25.60,25.33,99.14,40.35,45.85,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
7,25.82,25.49,100.00,40.63,45.45,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
8,25.60,25.33,99.14,40.35,43.91,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
9,25.82,25.49,100.00,40.63,44.35,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [34]:
df.to_csv('~/project/MIMIC-III/Model/Output/FINAL/SVM/SVM_100%_output.csv', index=False)

### DataFrame을 CSV 파일로 저장

In [8]:
# df1 = pd.read_csv("~/project/MIMIC-III/Model/Output/SVM/SVM(50%).csv")

# df2 = pd.read_csv("~/project/MIMIC-III/Model/Output/SVM(10%)_1.csv")
# # df3 = pd.read_csv("~/project/MIMIC-III/Model/Output/SVM(50%)_3.csv")
# # df4 = pd.read_csv("~/project/MIMIC-III/Model/Output/SVM(50%)_4.csv")
# # df5 = pd.read_csv("~/project/MIMIC-III/Model/Output/SVM(50%)_5.csv")
# # df6 = pd.read_csv("~/project/MIMIC-III/Model/Output/SVM(50%)_6.csv")
# merged_df = pd.concat([df1, df2], ignore_index=True)
# merged_df


In [9]:
# # 제거할 행의 조건을 설정
# condition = merged_df['Recall'] == 24.14  # 예제에서는 'A' 열 값이 3인 행을 제거하겠습니다.

# # 조건을 만족하는 행 제거
# merged_df = merged_df[~condition]
# merged_df = merged_df.reset_index(drop=True)
# merged_df

In [10]:
# merged_df.to_csv('~/project/MIMIC-III/Model/Output/SVM/SVM_50%_output.csv', index=False)  # index를 저장하지 않으려면 index=False로 설정
